In [ ]:
# Libraries
import networkx as nx
import matplotlib.pyplot as plt
import csv
import time
import heapq
import math
from math import atan2, sqrt, radians, sin, cos

In [ ]:
# A class for reading the files.
class reading:
    city_coordinates = {}
    adjacency_graph = {}

    def reading_coordinates(self):
        with open('/content/coordinates.csv', 'r') as file:
            csv_reader = csv.reader(file) # Creating a CSV reader object using csv.reader.

            for row in csv_reader:
                city_name, latitude, longitude = row
                self.city_coordinates[city_name] = (float(latitude), float(longitude))

        #print(self.city_coordinates)

    def reading_adjacencies(self):
        with open('/content/Adjacencies.txt', 'r') as file_2:
            for row in file_2:
                city_2, city_1 = row.strip().split()

                if city_1 not in self.adjacency_graph:
                    self.adjacency_graph[city_1]= []

                if city_2 not in self.adjacency_graph:
                    self.adjacency_graph[city_2] = []
                self.adjacency_graph[city_1].append(city_2)
                self.adjacency_graph[city_2].append(city_1)

readers = reading()
readers.reading_coordinates()
readers.reading_adjacencies()

In [ ]:
class undirected_search: # utilized lot of the code from the first iteration. Turned it into class and understood it better. More readable.

    def __init__(self, adjacency_graph):
        self.graph = adjacency_graph

    def breadth_first_search(self, start_city, end_city):
        visited_cities = set()
        queue = [[start_city, []]]

        while queue:
            current_city, path_city = queue.pop(0)
            if current_city == end_city:
                return path_city + [current_city]
            if current_city not in visited_cities:
                visited_cities.add(current_city)
                for adjacent_city in self.graph.get(current_city, []):
                    if adjacent_city not in visited_cities:
                        queue.append([adjacent_city, path_city + [current_city]])
        return None

    def depth_first_search(self, start_city, end_city):
        visited_cities = set()
        stack = [[start_city, []]]

        while stack:
            current_city, path_city = stack.pop()
            if current_city == end_city:
                return path_city + [current_city]
            if current_city not in visited_cities:
                visited_cities.add(current_city)
                for adjacent_city in self.graph.get(current_city, []):
                    if adjacent_city not in visited_cities:
                        stack.append((adjacent_city, path_city + [current_city]))
        return None

    def iterative_depth_first_search(self, start_city, end_city):
        depth = 0
        while True:
            result = self.depth_first_search_recursive(start_city, end_city, [], depth)
            if result is not None:
                return result
            depth += 1

    def depth_first_search_recursive(self, current_city, end_city, path_city, depth):
        if current_city == end_city:
            return path_city + [current_city]
        if depth == 0:
            return None
        for adjacent_city in self.graph.get(current_city, []):
            if adjacent_city not in path_city:
                result = self.depth_first_search_recursive(adjacent_city, end_city, path_city + [current_city], depth - 1)
                if result is not None:
                    return result
        return None

In [ ]:
class heuristic_search(user_interface): # utilized lot of the code from the first iteration. Turned it into class and understood it better. More readable.
    def __init__(self, adjacency_graph):
        self.graph = adjacency_graph
        self.city_coordinates = readers.city_coordinates

    def best_first_search(self, start_city, end_city):
        visited_city = set()
        heap = [(self.distance(self.city_coordinates[start_city], self.city_coordinates[end_city]), start_city, [])]

        while heap:
            _, current_city, path_city = heapq.heappop(heap)
            if current_city == end_city:
                return path_city + [current_city]
            if current_city not in visited_city:
                visited_city.add(current_city)
                for adjacent_city in self.graph.get(current_city, []):
                    if adjacent_city not in visited_city:
                        heapq.heappush(heap,(self.distance(self.city_coordinates[adjacent_city], self.city_coordinates[end_city]), adjacent_city, path_city + [current_city]))
        return None

    def a_star_search(self, start_city, end_city):
        visited_cities = set()
        heap = [(self.distance(self.city_coordinates[start_city], self.city_coordinates[end_city]), 0, start_city, [])]
        while heap:
            f, g, current_city, path_cities = heapq.heappop(heap)
            if current_city == end_city:
                return path_cities + [current_city]
            if current_city not in visited_cities:
                visited_cities.add(current_city)
                for adjacent_city in self.graph.get(current_city, []):
                    if adjacent_city not in visited_cities:
                        new_g = g + 1
                        h = self.distance(self.city_coordinates[adjacent_city], self.city_coordinates[end_city])
                        new_f = new_g + h
                        heapq.heappush(heap, (new_f, new_g, adjacent_city, path_cities + [current_city]))
        return None


In [ ]:
class user_interface: # utilized lot of the code from the first iteration. Turned it into class and understood it better. More readable.
    def __init__(self, city_coordinates, adjacency_graph):
        self.city_coordinates = city_coordinates
        self.adjacency_graph = adjacency_graph
        for city in self.city_coordinates:
            print(city)

    def distance(self, coordinate_1, coordinate_2):
            latitude_1, longitude_1 = coordinate_1
            latitude_2, longitude_2 = coordinate_2

            radius = 3958.8
            difference_latitude = radians(latitude_2 - latitude_1)
            difference_longitude = radians(longitude_2- longitude_1)

            eqn = sin(difference_latitude/2)**2 + cos(radians(latitude_1)) * cos(radians(latitude_2)) * sin(difference_longitude/2)**2
            distance = 2*atan2(sqrt(eqn),sqrt(1-eqn))
            return radius * distance

    def interface(self):
        while True:

            self.start_city = input("Enter Start City: ")
            self.end_city = input("Enter Goal City: ")
            if self.start_city not in self.city_coordinates or self.end_city not in self.city_coordinates:
                print("INVALID CITY!")
                continue

            print("\n")
            print("[1] Breadth-first")
            print("[2] Depth-first")
            print("[3] ID-FS")
            print("[4] Best-first")
            print("[5] A*")
            userInput = int(input("Select Search Algorithm [1] — [5]: "))

            start_time = time.time()

            match userInput:
                case 1:
                    path = undirected_search(self.adjacency_graph).breadth_first_search(self.start_city, self.end_city)
                case 2:
                    path = undirected_search(self.adjacency_graph).depth_first_search(self.start_city, self.end_city)
                case 3:
                    path = undirected_search(self.adjacency_graph).iterative_depth_first_search(self.start_city, self.end_city)
                case 4:
                    path = heuristic_search(self.adjacency_graph).best_first_search(self.start_city, self.end_city)
                case 5:
                    path = heuristic_search(self.adjacency_graph).a_star_search(self.start_city, self.end_city)
                case _:
                    print("Invalid Input!")
                    continue

            end_time = time.time() # Get time for program execution.

            if path:
                print("\n")
                print("Route:", path)
                total_distance = sum(self.distance(self.city_coordinates[path[i]], self.city_coordinates[path[i+1]]) for i in range(len(path)-1))
                print("Total Distance:", total_distance, "Miles")
                print("Execution Time:", end_time - start_time, "seconds")
                #print("Memory Used: ", self.)
            else:
                print("No Route Found")

            self.choice = input("\nSearch Again? [Yes or No] ")
            if self.choice.lower() == 'no':
                break

interface = user_interface(readers.city_coordinates, reading.adjacency_graph)
interface.interface()

Abilene
Andover
Anthony
Argonia
Attica
Augusta
Bluff_City
Caldwell
Cheney
Clearwater
Coldwater
Derby
El_Dorado
Emporia
Florence
Greensburg
Harper
Haven
Hays
Hillsboro
Hutchinson
Junction_City
Kingman
Kiowa
Leon
Lyons
Manhattan
Marion
Mayfield
McPherson
Medicine_Lodge
Mulvane
Newton
Oxford
Pratt
Rago
Salina
Sawyer
South_Haven
Topeka
Towanda
Viola
Wellington
Wichita
Winfield
Zenda
Enter Start City: Pratt
Enter Goal City: Viola


[1] Breadth-first
[2] Depth-first
[3] ID-FS
[4] Best-first
[5] A*
Select Search Algorithm [1] — [5]: 5


Route: ['Pratt', 'Cheney', 'Mulvane', 'South_Haven', 'Caldwell', 'Argonia', 'Rago', 'Viola']
Total Distance: 192.08385479265328 Miles
Execution Time: 0.00024247169494628906 seconds

Search Again? [Yes or No] No
